# Parte del video
## 1a. 
- Desarrollar una aplicación que lleve a cabo transformaciones de la imagen en tiempo real a través de una interfaz basada en trackbars o equivalente
- Hacer traslaciones. Es necesario indicar la magnitud de la traslación en X y en Y.
- Hacer rotaciones. Es necesario indicar el centro de giro y ángulo de giro.
- Hacer escalados uniformes y no uniformes. Es necesario indicar los factores de
escala.


In [3]:
import cv2 as cv
import numpy as np

# --- Abrir cámara (0 = webcam por defecto) ---
cap = cv.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("No se pudo abrir la cámara.")

# Leer un frame inicial para obtener dimensiones
ret, frame = cap.read()
if not ret:
    raise RuntimeError("No se pudo leer frame de la cámara.")

h, w = frame.shape[:2]

cv.namedWindow("Transformaciones")

def nothing(x):
    pass

# --- Crear trackbars ---
# Traslación
cv.createTrackbar("Tx", "Transformaciones", w // 2, w, nothing)   # [-w/2, w/2]
cv.createTrackbar("Ty", "Transformaciones", h // 2, h, nothing)   # [-h/2, h/2]

# Rotación
cv.createTrackbar("Angulo", "Transformaciones", 0, 360, nothing)
cv.createTrackbar("Cx", "Transformaciones", w // 2, w, nothing)
cv.createTrackbar("Cy", "Transformaciones", h // 2, h, nothing)

# Escalado
cv.createTrackbar("EscalaX", "Transformaciones", 100, 200, nothing)
cv.createTrackbar("EscalaY", "Transformaciones", 100, 200, nothing)

# Modo escalado: 0 = no uniforme, 1 = uniforme
cv.createTrackbar("Uniforme", "Transformaciones", 0, 1, nothing)

print("👉 Usa los sliders para transformar el video en tiempo real.")
print("Pulsa ESC para salir.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # --- Leer parámetros ---
    tx = cv.getTrackbarPos("Tx", "Transformaciones") - w // 2
    ty = cv.getTrackbarPos("Ty", "Transformaciones") - h // 2
    angle = cv.getTrackbarPos("Angulo", "Transformaciones")
    cx = cv.getTrackbarPos("Cx", "Transformaciones")
    cy = cv.getTrackbarPos("Cy", "Transformaciones")
    sx = cv.getTrackbarPos("EscalaX", "Transformaciones") / 100.0
    sy = cv.getTrackbarPos("EscalaY", "Transformaciones") / 100.0
    uniforme = cv.getTrackbarPos("Uniforme", "Transformaciones")

    # --- Traslación ---
    T = np.float32([[1, 0, tx],
                    [0, 1, ty]])
    img_T = cv.warpAffine(frame, T, (w, h))

    # --- Rotación respecto a (cx, cy) ---
    M_translate1 = np.float32([[1, 0, -cx],
                               [0, 1, -cy]])
    M_rotate = cv.getRotationMatrix2D((0, 0), angle, 1.0)
    M_translate2 = np.float32([[1, 0, cx],
                               [0, 1, cy]])

    M1 = np.vstack([M_translate1, [0, 0, 1]])
    M2 = np.vstack([M_rotate, [0, 0, 1]])
    M3 = np.vstack([M_translate2, [0, 0, 1]])
    M_rot = M3 @ M2 @ M1
    img_R = cv.warpAffine(img_T, M_rot[:2], (w, h))

    # --- Escalado ---
    if uniforme:
        s = sx
        S = np.float32([[s, 0, 0],
                        [0, s, 0]])
    else:
        S = np.float32([[sx, 0, 0],
                        [0, sy, 0]])
    img_final = cv.warpAffine(img_R, S, (w, h))

    # --- Dibujar puntos de referencia ---
    img_display = img_final.copy()

    # Punto de rotación (rojo)
    cv.circle(img_display, (cx, cy), 5, (0, 0, 255), -1)
    cv.putText(img_display, f"({cx},{cy})", (cx + 10, cy - 10),
               cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Punto de traslación aplicado (verde)
    tx_point, ty_point = w // 2 + tx, h // 2 + ty
    cv.circle(img_display, (tx_point, ty_point), 5, (0, 255, 0), -1)
    cv.putText(img_display, f"({tx_point},{ty_point})", (tx_point + 10, ty_point - 10),
               cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Mostrar resultado
    cv.imshow("Transformaciones", img_display)

    if cv.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv.destroyAllWindows()


👉 Usa los sliders para transformar el video en tiempo real.
Pulsa ESC para salir.


## 1b. 
- Dada una imagen trazar una ventana de proyección y proyectar la imagen.

In [6]:
import cv2
import numpy as np

# --- Variables globales ---
src_points = []
dst_points = []
phase = 0   # 0 = seleccionando src, 1 = seleccionando dst
frame_ref = None
homography_ready = False
H = None
dst_polygon = None

def ordenar_cuatro_puntos(pts):
    """ Ordena 4 puntos en el orden: TL, TR, BR, BL """
    pts = np.array(pts, dtype=np.float32)
    s = pts.sum(axis=1)
    diff = np.diff(pts, axis=1)

    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]
    tr = pts[np.argmin(diff)]
    bl = pts[np.argmax(diff)]
    return np.array([tl, tr, br, bl], dtype=np.float32)

def mouse_callback(event, x, y, flags, param):
    global src_points, dst_points, phase, frame_ref, H, homography_ready, dst_polygon

    if event == cv2.EVENT_LBUTTONDOWN:
        if phase == 0 and len(src_points) < 4:
            src_points.append([x, y])
            if len(src_points) == 4:
                print("👉 Selecciona ahora los 4 puntos destino (rojo).")
                phase = 1

        elif phase == 1 and len(dst_points) < 4:
            dst_points.append([x, y])
            if len(dst_points) == 4:
                print("✅ Calculando proyección...")
                src = ordenar_cuatro_puntos(src_points)
                dst = ordenar_cuatro_puntos(dst_points)
                H, _ = cv2.findHomography(src, dst)
                dst_polygon = np.int32(dst)
                homography_ready = True

def main():
    global frame_ref, H, homography_ready, dst_polygon

    # --- Abrir video (0 = webcam, o pon "video.mp4") ---
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise RuntimeError("No se pudo abrir el video/cámara.")

    # Capturar primer frame para selección inicial
    ret, frame_ref = cap.read()
    if not ret:
        raise RuntimeError("No se pudo leer el primer frame.")

    cv2.namedWindow("Proyeccion interactiva")
    cv2.setMouseCallback("Proyeccion interactiva", mouse_callback)

    print("👉 Haz click en 4 puntos fuente (verde), luego en 4 puntos destino (rojo).")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        output = frame.copy()

        if homography_ready and H is not None:
            h, w = frame.shape[:2]
            warped = cv2.warpPerspective(frame, H, (w, h))

            # Crear máscara de la zona destino
            mask = np.zeros((h, w), dtype=np.uint8)
            cv2.fillPoly(mask, [dst_polygon], 255)

            # Fondo negro y copia en la zona destino
            result = np.zeros_like(frame)
            cv2.copyTo(warped, mask, result)

            output = result

        # --- Dibujar puntos seleccionados siempre encima ---
        for (x, y) in src_points:
            cv2.circle(output, (int(x), int(y)), 6, (0, 255, 0), -1)  # verde
        for (x, y) in dst_points:
            cv2.circle(output, (int(x), int(y)), 6, (0, 0, 255), -1)  # rojo

        cv2.imshow("Proyeccion interactiva", output)

        key = cv2.waitKey(30) & 0xFF
        if key == 27:  # ESC
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


👉 Haz click en 4 puntos fuente (verde), luego en 4 puntos destino (rojo).
👉 Selecciona ahora los 4 puntos destino (rojo).
✅ Calculando proyección...
